In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import glob
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision.utils import make_grid
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
image_path = '/Users/patricknaylor/Desktop/Field_Detection/Images/Masked/'
torch.__version__
device = torch.device('mps')



/Users/patricknaylor/miniconda3/envs/field_detect/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/patricknaylor/miniconda3/envs/field_detect/lib/python3.10/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN2at4_ops19empty_memory_format4callEN3c108ArrayRefIxEENS2_8optionalINS2_10ScalarTypeEEENS5_INS2_6LayoutEEENS5_INS2_6DeviceEEENS5_IbEENS5_INS2_12MemoryFormatEEE
  Referenced from: <F0D48035-EF9E-3141-9F63-566920E60D7C> /Users/patricknaylor/miniconda3/envs/field_detect/lib/python3.10/site-packages/torchvision/image.so
  Expected in:     <B8EF5F9B-D0C8-3F96-967D-FE1E5F7F9E51> /Users/patricknaylor/miniconda3/envs/field_detect/lib/python3.10/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:

images_paths = glob.glob(image_path + '*.jpg')
mask_paths = glob.glob(image_path + '*.npy')
scale_percent = 10

images = []
masks = []
for (img, msk) in zip(images_paths, mask_paths):
    image = cv2.cvtColor(cv2.imread(img), cv2.COLOR_BGR2RGB)
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    images.append(cv2.resize(image, (width, height), interpolation = cv2.INTER_AREA))
    mask = np.load(msk).T
    mask = np.expand_dims(mask[25: -25, 25: -25], axis=2)
    mask = np.resize(mask, (36, 64, 1))
    masks.append(mask)


In [3]:
images_arr = np.stack(images)
masks_arr = np.stack(masks)


In [4]:
images_arr = images_arr.astype(np.uint8)
masks_arr = masks_arr.astype(np.uint8)

In [5]:
class imageDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        image = self.X[index]
        mask = self.y[index]
        y = torch.tensor(mask)
        X = self.transform(image)
        return [X, y]

    transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()])

    


    

In [6]:
print(np.shape(masks_arr))

(2753, 36, 64, 1)


In [7]:
batch_size = 8
transformed_dataset = imageDataset(X=images_arr, y=masks_arr)
train_set, val_set = torch.utils.data.random_split(transformed_dataset, [2000, 753])
train_dl = DataLoader(train_set, batch_size, shuffle=True)
val_dl = DataLoader(val_set, batch_size, shuffle=True)


In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5, padding='same')
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(int(6*72*128*.25), int(2*72*128*.25))
        self.fc2 = nn.Linear(int(2*72*128*.25), int(72*128*.25))
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [9]:
def cross_entropy(output, target):
    target = torch.flatten(target, 1)
    loss = -np.sum(target*np.log(output))
    return loss/float(output.shape[0])

model = Model().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [10]:
num_epochs = 5
n_total_steps = len(train_dl)
for epoch in range(num_epochs):
    running_loss = 0.0
    sum = 0
    for i, (input, target) in enumerate(train_dl):
        input = input.to(device)
        target = torch.flatten(target, 1).to(device)

        optimizer.zero_grad()

        outputs = model(input)
        loss = criterion(outputs, target.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        sum += 1

        if (i+1) % 20 == 0:
            print (f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{n_total_steps}], Loss: {loss.item():.4f}')



Epoch [1/5], Step [20/250], Loss: 0.6918
Epoch [1/5], Step [40/250], Loss: 0.6874
Epoch [1/5], Step [60/250], Loss: 0.6830
Epoch [1/5], Step [80/250], Loss: 0.6769
Epoch [1/5], Step [100/250], Loss: 0.6703
Epoch [1/5], Step [120/250], Loss: 0.6624
Epoch [1/5], Step [140/250], Loss: 0.6505
Epoch [1/5], Step [160/250], Loss: 0.6337
Epoch [1/5], Step [180/250], Loss: 0.6154
Epoch [1/5], Step [200/250], Loss: 0.5835
Epoch [1/5], Step [220/250], Loss: 0.5524
Epoch [1/5], Step [240/250], Loss: 0.5090
Epoch [2/5], Step [20/250], Loss: 0.3856
Epoch [2/5], Step [40/250], Loss: 0.3048
Epoch [2/5], Step [60/250], Loss: 0.2077
Epoch [2/5], Step [80/250], Loss: 0.1519
Epoch [2/5], Step [100/250], Loss: 0.1035
Epoch [2/5], Step [120/250], Loss: 0.0770
Epoch [2/5], Step [140/250], Loss: 0.0606
Epoch [2/5], Step [160/250], Loss: 0.0389
Epoch [2/5], Step [180/250], Loss: 0.0297
Epoch [2/5], Step [200/250], Loss: 0.0214
Epoch [2/5], Step [220/250], Loss: 0.0244
Epoch [2/5], Step [240/250], Loss: 0.0154
